## Initialise

In [1]:
from IPython.core.display import HTML
HTML("""
<style>
.container {
    width: 100%;
}
</style>
""")

In [2]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import tools 
import initialise

In [3]:
train_data, test_data = initialise.load_data(reload_data=False)

Loading saved processed data
This non-commercial license of GraphLab Create is assigned to kevin.mcisaac@gmail.com and will expire on November 06, 2016. For commercial licensing options, visit https://dato.com/buy/.

2016-03-11 08:13:39,877 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1457644418.log



306  features in total


# Logistic Classifier
- Had to remove rows with "None". Got AUC of 0.667
- Removed low variance features, which made no difference
- Switched to l2_penalty, and increated AUC to 0.80

In [4]:
train_data_1 = train_data.dropna()
model_1= gl.logistic_classifier.create(train_data_1, target='TARGET', l1_penalty=.05, l2_penalty=0,
                             max_iterations=50)

WARNING: Detected extremely low variance for feature(s) 'var18 indicator 0', 'var34 indicator 0', 'var18 number 0', 'var34 number 0', 'var18 balance', 'var34 balance', 'var18 amortization amount delta 1Y3', 'var34 amortization amount delta 1Y3', 'var13 refund amount delta 1Y3', 'var33 refund amount delta 1Y3', 'var17 delta transfer amount out 1Y3', 'var33 delta transfer amount out 1Y3', 'var18 repayment amount ultima1', 'var34 repayment amount ultima1', 'var13 refund amount ultima1', 'var17 refund amount ultima1', 'var33 refund amount ultima1', 'var17 transfer amount in ultima1', 'var17 transfer amount out ultima1', 'var33 transfer amount in ultima1', 'var33 transfer amount out ultima1', 'var13 reimbursement number ultima1', 'var17 reimbursement number ultima1', 'var33 reimbursement number ultima1', 'var17 transfer number in ultima1', 'var17 number transfer out ultima1', 'var33 transfer number in ultima1', 'var33 number transfer out ultima1' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

Number of examples          : 71582

Number of classes           : 2

...

| 6         | 7        | 0.000002  | 9.323948     | 0.960186          | 0.961447            |

| 10        | 11       | 0.000002  | 11.182808    | 0.960186          | 0.961183            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [5]:
model_1.evaluate(train_data, metric='auc')

{'auc': 0.6669024290928265}

In [6]:
low_variance =['var18 indicator 0', 'var34 indicator 0', 'var18 number 0', 'var34 number 0', 'var18 balance', 'var34 balance', 'var18 amortization amount delta 1Y3', 'var34 amortization amount delta 1Y3', 'var13 refund amount delta 1Y3', 'var33 refund amount delta 1Y3', 'var17 delta transfer amount out 1Y3', 'var33 delta transfer amount out 1Y3', 'var18 repayment amount ultima1', 'var34 repayment amount ultima1', 'var13 refund amount ultima1', 'var17 refund amount ultima1', 'var33 refund amount ultima1', 'var17 transfer amount in ultima1', 'var17 transfer amount out ultima1', 'var33 transfer amount in ultima1', 'var33 transfer amount out ultima1', 'var13 reimbursement number ultima1', 'var17 reimbursement number ultima1', 'var33 reimbursement number ultima1', 'var17 transfer number in ultima1', 'var17 number transfer out ultima1', 'var33 transfer number in ultima1', 'var33 number transfer out ultima1']
print len(low_variance), "low variance features removed"
train_data_2 = train_data.remove_columns(low_variance).dropna()
print len(train_data_2), "vs", len(train_data)

28 low variance features removed
75402 vs 76020


In [7]:
model_2= gl.logistic_classifier.create(train_data_2, target='TARGET', l1_penalty=.05, l2_penalty=0,
                             max_iterations=50)

Logistic regression:

--------------------------------------------------------

Number of examples          : 71658

Number of classes           : 2

Number of feature columns   : 278

...

| 6         | 7        | 0.000002  | 7.776061     | 0.960144          | 0.962340            |

| 10        | 11       | 0.000002  | 9.521810     | 0.960102          | 0.962073            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [8]:
model_2.evaluate(train_data, metric='auc')

{'auc': 0.6674052109511878}

In [9]:
model_3= gl.logistic_classifier.create(train_data_2, target='TARGET', l1_penalty=.1, l2_penalty=0,
                             max_iterations=50, verbose=False)
model_3.evaluate(train_data, metric='auc')

{'auc': 0.6674639851719227}

In [10]:
model_3= gl.logistic_classifier.create(train_data_2, target='TARGET', l1_penalty=.5, l2_penalty=0,
                             max_iterations=50, verbose=False)
model_3.evaluate(train_data, metric='auc')

{'auc': 0.6669915193006765}

In [11]:
model_4= gl.logistic_classifier.create(train_data_2, target='TARGET', 
                                       l1_penalty=0, l2_penalty=0.1,
                             max_iterations=50, verbose=False)
model_4.evaluate(train_data, metric='auc')

{'auc': 0.8009340456713047}

# Boosted Trees Classifier
- Standard pre-preocessing gets an AUC of 0.9179

In [22]:
model_gbt = gl.boosted_trees_classifier.create(train_data, target='TARGET', 
                                               max_depth=8, max_iterations=20)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 72191

Number of classes           : 2

Number of feature columns   : 278

...

| 18        | 11.717429    | 0.963278          | 0.961348            |

| 19        | 12.319039    | 0.963416          | 0.961609            |

| 20        | 12.899234    | 0.9635            | 0.961609            |

+-----------+--------------+-------------------+---------------------+

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [23]:
model_gbt.evaluate(train_data, metric='auc')

{'auc': 0.9179671586155758}

In [12]:
model_gbt = gl.boosted_trees_classifier.create(train_data, target='TARGET', metric='AUC', 
                                               max_depth=8, max_iterations=50)

WARNING: Ignore unsupported tracking metric AUC

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 72143

Number of classes           : 2

...

| 40        | 22.597357    |

| 45        | 25.327082    |

| 50        | 28.135490    |

+-----------+--------------+

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

